In [1]:
import os
import sys
import psutil
import traci
import time

# Ensure SUMO_HOME is set
if "SUMO_HOME" in os.environ:
    sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

# Function to kill old SUMO processes before starting
def kill_sumo():
    for process in psutil.process_iter(attrs=["pid", "name"]):
        if "sumo" in process.info["name"].lower():
            print(f"Killing SUMO process {process.info['name']} (PID: {process.info['pid']})")
            os.kill(process.info["pid"], 9)

kill_sumo()

# Run simulation
sumoBinary = "sumo/sumo/bin/sumo"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

try:
    # Increase retries to prevent connection issues
    traci.start(sumoCmd, numRetries=20)

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        for veh_id in traci.vehicle.getIDList():
            try:
                # Access the battery level directly using traci.vehicle.getParameter
                battery_level = traci.vehicle.getParameter(veh_id, "device.battery.actualBatteryCapacity")
                print(f"Vehicle {veh_id} Battery Level: {battery_level} kWh")
            except traci.TraCIException as e:
                print(f"Error retrieving battery level for {veh_id}: {e}")
                
except traci.exceptions.FatalTraCIError as e:
    print(f"TraCI connection lost: {e}")

except BrokenPipeError:
    print("BrokenPipeError: SUMO crashed or closed unexpectedly.")

finally:
    time.sleep(1)
    try:
        if traci.connection._connections:
            print("Closing TraCI connection properly...")
            traci.close()
    except Exception as e:
        print(f"TraCI was already closed: {e}")


 Retrying in 1 seconds
***Starting server on port 56875 ***


Loading net-file from 'osm.net.xml.gz' ... done (71ms).
Loading additional-files from 'osm.poly.xml.gz' ... done (19ms).
Loading additional-files from 'charging-stations.add.xml' ... done (1ms).
Loading route-files incrementally from 'osm.passenger.trips.xml'
Loading route-files incrementally from 'ev-routes.rou.xml'
Loading done.
The battery device is active for vType 'veh_passenger' but no emission class is set. The emission class Energy/unknown will be used, please consider setting an explicit emission class!
The battery device is active for vType 'electric_car' but no emission class is set. The emission class Energy/unknown will be used, please consider setting an explicit emission class!
Simulation version v1_21_0+2163-fe398d3a3bc started with time: 0.00.
Vehicle EV_1 Battery Level: 17500.00 kWh
Vehicle veh0 Battery Level: 17500.00 kWh
Vehicle EV_1 Battery Level: 17498.37 kWh
Vehicle veh0 Battery Level: 17502.08 kWh
Vehicle EV_1 Battery Level: 17495.95 kWh
Vehicle veh0 Battery Lev